In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [6]:
X_train, y_train = torch.load('train_data.pt')
X_test, y_test = torch.load('test_data.pt')

In [7]:
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_seq = self.X.iloc[idx].values
        y_val = self.y.iloc[idx]
        
        X_tensor = torch.tensor([item for item in X_seq], dtype=torch.float32)
        y_tensor = torch.tensor(y_val, dtype=torch.float32)
        
        return X_tensor, y_tensor

In [8]:
train_dataset = SequenceDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = SequenceDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
for X_batch, y_batch in train_loader:
    print(X_batch, y_batch)
    break

tensor([[[ 3.6937e+01,  3.6934e+01,  3.7045e+01,  ...,  3.7384e+01,
           3.7153e+01,  3.7264e+01],
         [ 1.7899e+02,  1.7928e+02,  1.7928e+02,  ...,  1.7872e+02,
           1.8000e+02,  1.7986e+02],
         [ 1.8200e+02,  1.8200e+02,  1.8000e+02,  ...,  2.7800e+02,
           2.7800e+02,  2.7800e+02],
         ...,
         [ 2.3742e-01,  2.3742e-01,  2.3742e-01,  ...,  2.3742e-01,
           2.3742e-01,  2.3742e-01],
         [ 1.1404e+02,  1.1404e+02,  1.1404e+02,  ...,  1.1404e+02,
           1.1404e+02,  1.1404e+02],
         [ 3.3030e+01,  3.3029e+01,  3.3029e+01,  ...,  3.2964e+01,
           3.2964e+01,  3.2963e+01]],

        [[ 9.7686e+00,  1.0479e+01,  1.0607e+01,  ...,  1.3585e+01,
           1.2887e+01,  1.1781e+01],
         [ 3.5148e+02,  3.5487e+02,  3.5401e+02,  ...,  3.5569e+02,
           3.5811e+02,  3.5752e+02],
         [ 1.2900e+02,  1.3100e+02,  1.3300e+02,  ...,  1.8700e+02,
           1.8900e+02,  1.9300e+02],
         ...,
         [ 1.9553e-01,  1